# √çNDICES e PARTICIONAMENTO

Na aula anterior estudamos os conceitos de **depend√™ncias funcionais** e **normaliza√ß√£o**. O conhecimento das formas normais propicia verificar a possibilidade de realizar altera√ß√µes no design das bases de dados, com o objetivo de **evitar repeti√ß√µes** e **recuperar informa√ß√µes de forma f√°cil**.

Entretanto, mesmo obedecendo (ou n√£o) √†s formas normais, em certas situa√ß√µes a performance de nossas queries em produ√ß√£o se torna sofr√≠vel. Nesta aula, veremos alguns recursos que podemos utilizar para melhorar o desempenho das consultas produzidas.

Este √© um tema bastante relevante. Um engenheiro sem o devido conhecimento pode recomendar, por exemplo, a compra de mais servidores (f√≠sicos ou na AWS), quando a aplica√ß√£o dos conceitos vistos nesta aula poderiam gerar ganhos de m√∫ltiplas vezes no tempo de execu√ß√£o de queries!

## Instala√ß√£o da base

Vamos utilizar a base de dados sint√©tica dispon√≠vel no Blackboard. Execute o script `script_elet_001.sql` para criar a base de dados. Este script apenas faz a **DDL**, os dados ger√£o gerados de forma aleat√≥ria neste notebook.

## Import das bibliotecas

Vamos realizar o import das bibliotecas.

In [1]:
import mysql.connector
import os
import random
import numpy as np
import pandas as pd
from functools import partial
from datetime import datetime, timedelta
from dotenv import load_dotenv
from IPython.display import clear_output

E vamos criar nosso HELPER de conex√£o com o banco! Perceba que, uma vez configurado o `.env` n√£o precisaremos mais informar usu√°rios, senhas e URLs!

In [3]:
load_dotenv(override=True)

def get_connection_helper():

    def run_db_query(connection, query, args=None, verbose=True):
        with connection.cursor() as cursor:
            if verbose:
                print("Executando query:")
            cursor.execute(query, args)
            for result in cursor:
                if verbose:
                    print(result)

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        port=int(os.getenv("MD_DB_PORT", 3306)),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database="eletrobeer",
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

## Gerar dados para a base

Vamos gerar alguns valores aleat√≥rios e inserir na base de dados `eletrobeer`.

Primeiro, defina a quantidade de linhas a serem inseridas e demais vari√°veis de ambiente:

In [4]:
QTDE_PED = 6000000
BATCH_SIZE = 200000

START_DATE = datetime(2015, 1, 1)
END_DATE = datetime(2022, 12, 31)

CIDADES_UF = np.array([
    ["S√£o Paulo", "SP"],
    ["Campinas", "SP"],
    ["Ribeir√£o Preto", "SP"],
    ["S√£o Roque", "SP"],
    ["Rio de Janeiro", "RJ"],
    ["Maca√©", "RJ"],
    ["Angra dos Reis", "RJ"],
])
NUM_CIDADES = len(CIDADES_UF)
DATE_DELTA_DAYS = (END_DATE - START_DATE).days

Vamos definir algumas fun√ß√µes auxiliares

In [5]:
def gerar_lote_pedidos(start_id, batch_size):
    """Gera um lote de pedidos com dados aleat√≥rios."""
    ids_pedido = np.arange(start_id, start_id + batch_size, dtype=np.int32)
    ids_cliente = np.random.randint(1000, 10001, size=batch_size, dtype=np.int32)
    qtde_itens = np.random.randint(1, 201, size=batch_size, dtype=np.int32)
    valor_total = np.random.uniform(15.0, 300000.0, size=batch_size)
    
    random_days = np.random.randint(0, DATE_DELTA_DAYS + 1, size=batch_size)
    datas_criacao = (
        pd.to_datetime(START_DATE) + pd.to_timedelta(random_days, unit='D')
    ).strftime("%Y-%m-%d").to_numpy()
    
    indices_cidade = np.random.randint(0, NUM_CIDADES, size=batch_size)
    cidades = CIDADES_UF[indices_cidade, 0]
    ufs = CIDADES_UF[indices_cidade, 1]

    pedidos = list(zip(
        ids_pedido.tolist(),
        ids_cliente.tolist(),
        datas_criacao,
        qtde_itens.tolist(),
        valor_total.tolist(),
        cidades,
        ufs
    ))
    
    return pedidos


def insert_pedidos(connection, pedidos):
    """Insere um lote de pedidos na tabela `pedido`."""
    with connection.cursor() as cursor:
        sql = """INSERT INTO eletrobeer.pedido
        (id_pedido, id_cliente, data_criacao, qtde_itens, valor_total, cidade_entrega, uf_entrega)
        VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        cursor.executemany(sql, pedidos)

Garantir que a tabela `pedido` est√° vazia

In [6]:
db("TRUNCATE eletrobeer.pedido")

Executando query:


Ent√£o, geramos os dados aleat√≥rios.

**<span style="color:red">Aten√ß√£o</span>**: Este processo pode demorar alguns minutos! A cada aproximadamente 5 segundos, `BATCH_SIZE` inser√ß√µes devem ser realizadas, com o retorno da mensagem `Completou 800,000 pedidos (Lote 1 de 30)` e assim por diante!

**Dica:** enquanto executa a pr√≥xima c√©lula, apenas por curiosidade, leia o seguinte material https://dev.mysql.com/doc/refman/8.0/en/optimizing-innodb-bulk-data-loading.html.

In [7]:
num_batches = (QTDE_PED + BATCH_SIZE - 1) // BATCH_SIZE

print(f"Iniciando a gera√ß√£o e processamento de {QTDE_PED:,} pedidos em {num_batches} lotes de {BATCH_SIZE:,}...")

for i in range(num_batches):
    start_id = i * BATCH_SIZE + 1
    
    current_batch_size = min(BATCH_SIZE, QTDE_PED - (start_id - 1))
    
    pedidos = gerar_lote_pedidos(start_id, current_batch_size)
    
    if connection:
        insert_pedidos(connection, pedidos)
        
    if (i + 1) * BATCH_SIZE % BATCH_SIZE == 0:
        clear_output(wait=True)
        print(f"Completou {(i + 1) * BATCH_SIZE:,} pedidos (Lote {i+1} de {num_batches})")

print(f"Processamento de {QTDE_PED:,} pedidos conclu√≠do.")

Completou 6,000,000 pedidos (Lote 30 de 30)
Processamento de 6,000,000 pedidos conclu√≠do.


E vamos fazer `commit` para garantir que os dados foram salvos!

In [8]:
connection.commit()

## Consultando a base

Vamos fazer algumas consultas. Como nosso objetivo √© avaliar a performance das queries, precisamos analisar o tempo que cada query necessita para executar.

Uma op√ß√£o √© fazer o c√°lculo direto no Python:

In [9]:
import time

start_time = time.time() # get current time

db("SELECT COUNT(*) FROM pedido")

end_time = time.time() # get current time again

time_spent = end_time - start_time # calculate time spent

print("Time spent:", time_spent, "seconds")

Executando query:
(6000000,)
Time spent: 0.3182504177093506 seconds


Entretanto, desta forma estamos considerando o tempo total, incluindo o tempo gasto pelas fun√ß√µes do pr√≥prio Python. Talvez n√£o seja um tempo significativo, mas como poderia ser, melhor evitar e considerar o tempo isolado: apenas o que foi gasto de fato para a query executar (ap√≥s ter sido recebida pelo RDBMS MySQL).

Para isto, vamos ativar **profiling**:

In [10]:
db("SET profiling = 1;", verbose=False)

**Obs**: quando quiser desativar, utilize

```mysql
SET profiling = 0;
```

Ent√£o podemos executar algum **SQL**

In [11]:
db("SELECT COUNT(*) FROM pedido")

Executando query:
(6000000,)


Um outro exemplo de **SQL**

In [12]:
db("SELECT * FROM pedido ORDER BY cidade_entrega ASC LIMIT 1")

Executando query:
(7, 8407, datetime.date(2015, 10, 16), 63, Decimal('59811.99'), 'Angra dos Reis', 'RJ')


E ver, no segundo elemento de cada tupla, qual o tempo gasto para a query ser executada.

In [13]:
db("SHOW PROFILES;")

Executando query:
(1, 0.34389675, 'SELECT COUNT(*) FROM pedido')
(2, 3.102552, 'SELECT * FROM pedido ORDER BY cidade_entrega ASC LIMIT 1')


Ent√£o, podemos executar novas queries e ver o seu tempo consumido:

**Dica**: aqui, `verbose=False` executar√° a query, mas n√£o exibir√° o resultado na sa√≠da. Podemos fazer assim quando nosso interesse √© em apenas ter o tempo da query e n√£o o resultado

In [14]:
sql = "SELECT COUNT(*) as QTDE_LINHAS FROM pedido"

db(sql, verbose=False)
db("SHOW PROFILES;")

Executando query:
(1, 0.34389675, 'SELECT COUNT(*) FROM pedido')
(2, 3.102552, 'SELECT * FROM pedido ORDER BY cidade_entrega ASC LIMIT 1')
(3, 0.27147825, 'SELECT COUNT(*) as QTDE_LINHAS FROM pedido')


Utilize `SET PROFILING_HISTORY_SIZE = ???;`, trocando `???` pelo **n√∫mero de queries** que quer ver no resultado dos profiles, para limitar a exibi√ß√£o as √∫ltimas **n√∫mero de queries**.

In [15]:
db("SET PROFILING_HISTORY_SIZE = 2;", verbose=False)
db("SHOW PROFILES;")

Executando query:
(3, 0.27147825, 'SELECT COUNT(*) as QTDE_LINHAS FROM pedido')
(4, 0.00035975, 'SET PROFILING_HISTORY_SIZE = 2')


Vamos deixar configurado para `6`. Voc√™ pode alterar quando quiser!

In [16]:
db("SET PROFILING_HISTORY_SIZE = 6;", verbose=False)
db("SHOW PROFILES;")

Executando query:
(3, 0.27147825, 'SELECT COUNT(*) as QTDE_LINHAS FROM pedido')
(4, 0.00035975, 'SET PROFILING_HISTORY_SIZE = 2')
(5, 0.00020825, 'SET PROFILING_HISTORY_SIZE = 6')


### Explorando novos exemplos

Vamos executar algumas consultas e verificar seus tempos de execu√ß√£o.

**Dica**: leia cada query e tente entender o que ela faz!

In [17]:
sql1 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega = "S√£o Paulo";"""

sql2 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega = "Rio de Janeiro";"""

sql3 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega IN ("Rio de Janeiro", "S√£o Paulo");"""

sql4 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega LIKE "S√£o%";"""

sql5 = r"""SELECT count(*) FROM pedido p
WHERE p.cidade_entrega LIKE "%de%";"""

sql6 = "SELECT * FROM pedido ORDER BY cidade_entrega DESC LIMIT 5"

db(sql1)
db(sql2)
db(sql3)
db(sql4)
db(sql5)
db(sql6)

db("SHOW PROFILES;")

Executando query:
(855743,)
Executando query:
(857084,)
Executando query:
(1712827,)
Executando query:
(1712655,)
Executando query:
(857084,)
Executando query:
(11, 1333, datetime.date(2022, 7, 19), 52, Decimal('22675.44'), 'S√£o Roque', 'SP')
(12, 9949, datetime.date(2019, 5, 26), 76, Decimal('295677.71'), 'S√£o Roque', 'SP')
(4, 3963, datetime.date(2022, 1, 27), 185, Decimal('273296.47'), 'S√£o Roque', 'SP')
(28, 5633, datetime.date(2018, 1, 9), 94, Decimal('251777.12'), 'S√£o Roque', 'SP')
(6, 3824, datetime.date(2016, 8, 21), 135, Decimal('47693.05'), 'S√£o Roque', 'SP')
Executando query:
(6, 1.8067405, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega = "S√£o Paulo"')
(7, 1.48809075, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega = "Rio de Janeiro"')
(8, 2.0048515, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega IN ("Rio de Janeiro", "S√£o Paulo")')
(9, 1.46724825, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega LIKE "S√£o%"')
(10, 1.849854, 'SELECT coun

### MEGADADOS?!

Podemos perceber que esta base tem (deveria ter!!!) 4 milh√µes de linhas.

In [18]:
sql = "SELECT COUNT(*) as QTDE_LINHAS FROM pedido"

db(sql)

Executando query:
(6000000,)


No resultados do `SHOW PROFILES`, vemos que o tempo √© quase zero. Mas n√£o se engane, estamos mantendo um ambiente controlado, com apenas uma tabela e realizando queries simples o suficiente para entendermos o que est√° acontecendo.

Em uma situa√ß√£o do mercado profissional, prepare-se para trabalhar com milh√µes ou bilh√µes de linhas em dezenas ou centenas de tabelas, construindo queries de centenas e at√© milhares de linhas que rodam milhares ou milh√µes de vezes! As situa√ß√µes de depend√™ncia entre m√∫ltiplas colunas e tabelas gerar√° muitas situa√ß√µes onde os conceitos da aula poder√£o ser aplicados.

**Dica**: No MySQL Workbench, d√™ bot√£o direito na tabela / Table Inspector. Abra o explorer e confira o tamanho do arquivo contido no **Data path**! Novamente, em uma situa√ß√£o de mercado, espere gigabytes!

### √çndices

√çndices s√£o estruturas de dados que facilitam a localiza√ß√£o de informa√ß√£o no banco de dados.

**Obs**: Link para simular `BTREE` https://www.cs.usfca.edu/~galles/visualization/BTree.html

Para criar um √≠ndice, vamos utilizar a sintaxe:
```mysql
CREATE INDEX index_name [index_type] 
 ON tbl_name (index_col_name,...)
```

Por exemplo:

```mysql
-- Por padr√£o, o index ser√° BTREE
CREATE INDEX pedido_cidade_entrega_IDX
 ON eletrobeer.pedido (cidade_entrega);
```

√â importante lembrar que nem todos os engines suportam √≠ndice **HASH**. A engine padr√£o da vers√£o que utilizamos (**InnoDB**) n√£o suporta!

In [19]:
db("SHOW ENGINES;")

Executando query:
('ndbcluster', 'NO', 'Clustered, fault-tolerant tables', None, None, None)
('MEMORY', 'YES', 'Hash based, stored in memory, useful for temporary tables', 'NO', 'NO', 'NO')
('InnoDB', 'DEFAULT', 'Supports transactions, row-level locking, and foreign keys', 'YES', 'YES', 'YES')
('PERFORMANCE_SCHEMA', 'YES', 'Performance Schema', 'NO', 'NO', 'NO')
('MyISAM', 'YES', 'MyISAM storage engine', 'NO', 'NO', 'NO')
('FEDERATED', 'NO', 'Federated MySQL storage engine', None, None, None)
('ndbinfo', 'NO', 'MySQL Cluster system information storage engine', None, None, None)
('MRG_MYISAM', 'YES', 'Collection of identical MyISAM tables', 'NO', 'NO', 'NO')
('BLACKHOLE', 'YES', '/dev/null storage engine (anything you write to it disappears)', 'NO', 'NO', 'NO')
('CSV', 'YES', 'CSV storage engine', 'NO', 'NO', 'NO')
('ARCHIVE', 'YES', 'Archive storage engine', 'NO', 'NO', 'NO')


Vamos criar um √≠ndice na coluna `cidade_entrega` e repetir as queries.

Mas antes, vamos consultar se existe algum √≠ndice atualmente nesta tabela!

In [20]:
db("SHOW INDEX FROM pedido;")

Executando query:
('pedido', 0, 'PRIMARY', 1, 'id_pedido', 'A', 5751647, None, None, '', 'BTREE', '', '', 'YES', None)


Perceba que j√° existe um √≠ndice. Por que ele est√° a√≠ se ainda n√£o criamos nenhum?!

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

Sua resposta AQUI!

</div>

<a href="#" title="O √≠ndice foi criado para a coluna que √© chave prim√°ria! Isto √© feito por padr√£o, uma vez que a chave acaba sendo utilizada em diversas consultas e joins.">Pare o  mouse aqui para ver a resposta</a>

Ent√£o criamos o √≠ndice na coluna `cidade_entrega`:

In [21]:
db("CREATE INDEX pedido_cidade_entrega_IDX ON eletrobeer.pedido (cidade_entrega);")

Executando query:


E conferimos novamente os √≠ndices existentes

In [22]:
db("SHOW INDEX FROM pedido;")

Executando query:
('pedido', 0, 'PRIMARY', 1, 'id_pedido', 'A', 5751647, None, None, '', 'BTREE', '', '', 'YES', None)
('pedido', 1, 'pedido_cidade_entrega_IDX', 1, 'cidade_entrega', 'A', 6, None, None, 'YES', 'BTREE', '', '', 'YES', None)


Caso queira ver o t√≠tulo das coluas, execute o `SHOW INDEX FROM pedido;` direto no MySQL WorkBench!

Agora repetimos as queries. Compare o tempo necess√°rios para suas execu√ß√µes.

In [23]:
sql1 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega = "S√£o Paulo";"""

sql2 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega = "Rio de Janeiro";"""

sql3 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega IN ("Rio de Janeiro", "S√£o Paulo");"""

sql4 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega LIKE "S√£o%";"""

sql5 = r"""SELECT count(*) FROM pedido p
WHERE p.cidade_entrega LIKE "%de%";"""

sql6 = "SELECT * FROM pedido ORDER BY cidade_entrega DESC LIMIT 5"

db(sql1)
db(sql2)
db(sql3)
db(sql4)
db(sql5)
db(sql6)

db("SHOW PROFILES;")

Executando query:
(855743,)
Executando query:
(857084,)
Executando query:
(1712827,)
Executando query:
(1712655,)
Executando query:
(857084,)
Executando query:
(5999983, 5359, datetime.date(2022, 5, 8), 123, Decimal('143364.78'), 'S√£o Roque', 'SP')
(5999977, 5774, datetime.date(2022, 3, 6), 59, Decimal('242203.06'), 'S√£o Roque', 'SP')
(5999974, 1490, datetime.date(2020, 6, 7), 150, Decimal('23525.21'), 'S√£o Roque', 'SP')
(5999948, 9333, datetime.date(2015, 6, 7), 161, Decimal('264705.19'), 'S√£o Roque', 'SP')
(5999944, 3858, datetime.date(2022, 5, 19), 5, Decimal('214935.79'), 'S√£o Roque', 'SP')
Executando query:
(17, 0.3485355, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega = "S√£o Paulo"')
(18, 0.21028775, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega = "Rio de Janeiro"')
(19, 0.97835675, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega IN ("Rio de Janeiro", "S√£o Paulo")')
(20, 0.447218, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega LIKE "S√£o%"')

Compare o tempo **antes** *versus* **depois** da cria√ß√£o do √≠ndice. Apesar de n√£o estamos analisando uma amostra de tamanho 1 (CDados manda oi!), √© esperado que obtenha uma melhora de m√∫ltiplas vezes em algumas queries, mas em outras nem tanto. Voc√™ consegue explicar o por que?!

**Dica**: https://dev.mysql.com/doc/refman/8.0/en/index-btree-hash.html#btree-index-characteristics

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

Sua resposta AQUI!

</div>

#### Testando com outros campos e queries!

Vamos testar com com outros campos e queries!

In [24]:
db("SET PROFILING_HISTORY_SIZE = 4;", verbose=False)

In [25]:
sql1 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens = 8;"""

sql2 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens < 5;"""

sql3 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens BETWEEN 5 AND 20;"""

sql4 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens IN (8, 15, 20, 45);"""

db(sql1)
db(sql2)
db(sql3)
db(sql4)

db("SHOW PROFILES;")

Executando query:
(29999,)
Executando query:
(120394,)
Executando query:
(479855,)
Executando query:
(119857,)
Executando query:
(24, 1.77885075, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens = 8')
(25, 1.3654305, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens < 5')
(26, 1.29889725, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens BETWEEN 5 AND 20')
(27, 1.5270245, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens IN (8, 15, 20, 45)')


Conferindo os √≠ndices atuais

In [26]:
db("SHOW INDEX FROM pedido;")

Executando query:
('pedido', 0, 'PRIMARY', 1, 'id_pedido', 'A', 5751647, None, None, '', 'BTREE', '', '', 'YES', None)
('pedido', 1, 'pedido_cidade_entrega_IDX', 1, 'cidade_entrega', 'A', 6, None, None, 'YES', 'BTREE', '', '', 'YES', None)


Crie um index **BTREE** baseado na coluna `qtde_itens`

In [27]:
db("CREATE INDEX pedido_qtde_itens_IDX ON eletrobeer.pedido (qtde_itens);")

Executando query:


Conferindo os √≠ndices

In [28]:
db("SHOW INDEX FROM pedido;")

Executando query:
('pedido', 0, 'PRIMARY', 1, 'id_pedido', 'A', 5751647, None, None, '', 'BTREE', '', '', 'YES', None)
('pedido', 1, 'pedido_cidade_entrega_IDX', 1, 'cidade_entrega', 'A', 6, None, None, 'YES', 'BTREE', '', '', 'YES', None)
('pedido', 1, 'pedido_qtde_itens_IDX', 1, 'qtde_itens', 'A', 201, None, None, 'YES', 'BTREE', '', '', 'YES', None)


Repetindo as consultas

In [29]:
sql1 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens = 8;"""

sql2 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens < 5;"""

sql3 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens BETWEEN 5 AND 20;"""

sql4 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens IN (8, 15, 20, 45);"""

db(sql1)
db(sql2)
db(sql3)
db(sql4)

db("SHOW PROFILES;")

Executando query:
(29999,)
Executando query:
(120394,)
Executando query:
(479855,)
Executando query:
(119857,)
Executando query:
(31, 0.00726175, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens = 8')
(32, 0.0354705, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens < 5')
(33, 0.07621825, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens BETWEEN 5 AND 20')
(34, 0.0230045, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens IN (8, 15, 20, 45)')


Compare o tempo **antes** *versus* **depois** da cria√ß√£o do √≠ndice. Emocionante, n√£o?!

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

Sua resposta AQUI!

</div>

## Particionamento

Particionar √© dividir as tabelas de um banco de dados em partes menores.

Permite distribuir o banco de dados em v√°rios n√≥s ou HDs diferentes, aumentando o desempenho em situa√ß√µes de acesso concorrente intenso (que n√£o √© o nosso caso).

Leia mais em https://dev.mysql.com/doc/refman/8.0/en/partitioning-pruning.html

Veja um exemplo de particionamento:

In [37]:
sql1 = """
SELECT 
    YEAR(p.data_criacao), AVG(valor_total) AS media
FROM
    pedido p
WHERE YEAR(p.data_criacao) > 2020
GROUP BY YEAR(p.data_criacao)
ORDER BY YEAR(p.data_criacao) ASC;"""

db(sql1)

db("SHOW PROFILES;")

Executando query:
(2021, Decimal('149938.637044'))
(2022, Decimal('149919.921742'))
Executando query:
(39, 5.1065265, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')
(40, 2.16611075, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')
(41, 2.043404, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')
(42, 2.0785165, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')


Para separar por ano, precisaremos fazer com que a `data_criacao` seja parte da chave prim√°ria.

In [31]:
sql = """
ALTER TABLE pedido
DROP PRIMARY KEY;"""

db(sql)

Executando query:


In [32]:
sql = """
ALTER TABLE pedido
ADD PRIMARY KEY (id_pedido, data_criacao);"""

db(sql)

Executando query:


Ent√£o, separamos a tabela `pedido` em quatro parti√ß√µes

In [33]:
sql = """
ALTER TABLE pedido
PARTITION BY RANGE(YEAR(data_criacao))
(
    PARTITION p0 VALUES LESS THAN (2016),
    PARTITION p1 VALUES LESS THAN (2018),
    PARTITION p2 VALUES LESS THAN (2020),
    PARTITION p3 VALUES LESS THAN MAXVALUE
);"""

db(sql)

Executando query:


Executando a query novamente

In [38]:
sql1 = """
SELECT 
    YEAR(p.data_criacao), AVG(valor_total) AS media
FROM
    pedido p
WHERE YEAR(p.data_criacao) > 2020
GROUP BY YEAR(p.data_criacao)
ORDER BY YEAR(p.data_criacao) ASC;"""

db(sql1)

db("SHOW PROFILES;")

Executando query:
(2021, Decimal('149938.637044'))
(2022, Decimal('149919.921742'))
Executando query:
(40, 2.16611075, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')
(41, 2.043404, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')
(42, 2.0785165, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')
(43, 2.066346, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')


Anote abaixo suas considera√ß√µes sobre particionar tabelas!

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

Suas observa√ß√µes AQUI!

</div>

## Exerc√≠cios

**Exerc√≠cio 1**: Explique por que uma hash table √©:

**a)** Boa para buscas por valor exato?

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">
Uma hash table √© excelente para buscas por valor exato porque ela usa uma fun√ß√£o de hash para mapear cada chave diretamente para um √≠ndice na tabela. Isso permite acessar o valor em tempo constante m√©dio (O(1)), sem precisar percorrer todos os elementos.

</div>

**b)** Ruim para buscas por faixas de valor?

**Dicas**:
- Tente pensar por alguns minutos como as hash tables funcionam
- Se travar, pe√ßa ajuda aos professores ou pergunte ao ChatGPT.

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

As hash tables s√£o ruins para buscas por faixas de valor (por exemplo, ‚Äúvalores entre 10 e 20‚Äù), porque os elementos n√£o s√£o armazenados em ordem.
A fun√ß√£o de hash ‚Äúembaralha‚Äù as chaves, ent√£o n√£o h√° rela√ß√£o entre os √≠ndices e a ordem dos valores ‚Äî seria necess√°rio verificar todas as entradas da tabela, o que tem custo O(n).

</div>

<a href="#gab_ex1">Click para ver a resposta</a>

**Exerc√≠cio 2**: Por que os √≠ndices de bancos de dados relacionais utilizam `B-tree` e suas variantes ao inv√©s de uma √°rvore bin√°ria balanceada?

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

Os bancos de dados usam B-trees porque elas reduzem o n√∫mero de acessos a disco, mantendo os dados ordenados e balanceados, o que permite buscas r√°pidas e eficientes por faixas de valores.
√Årvores bin√°rias balanceadas s√£o menos eficientes nesse contexto, pois exigem muitos acessos a disco e n√£o s√£o otimizadas para intervalos.

</div>

**Exerc√≠cio 3**: O professor demonstrou a constru√ß√£o de uma `B-tree`. Pesquise o que s√£o as `B+-trees` e responda:

**Obs:**
- O `+` representa um **plus**. J√° o `-` √© s√≥ um tra√ßo separador!
- Voc√™ pode simular a vers√£o **plus** aqui https://www.cs.usfca.edu/~galles/visualization/BPlusTree.html

**a)** Qual a diferen√ßa entre a `B-tree` e a vers√£o plus?!

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

A B-tree guarda dados (ponteiros) em todos os n√≥s, enquanto a B+ tree guarda os dados somente nas folhas e mant√©m as folhas ligadas entre si, o que torna mais f√°cil buscar faixas de valores.

</div>

**b)** O MySQL utiliza `B-tree` ou `B+-trees`?

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

O MySQL utiliza B+ trees nos seus √≠ndices (como no InnoDB e MyISAM), pois elas s√£o mais eficientes para buscas por valor exato e por intervalos.

</div>

**Exerc√≠cio 4**: Explique por que uma B-tree √©:

**a)** Razo√°vel para buscas por valor exato?

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

Porque a B-tree √© balanceada ‚Äî a altura da √°rvore √© pequena ‚Äî ent√£o o banco consegue chegar rapidamente ao valor exato em tempo logar√≠tmico (O(log n)), sem percorrer todos os dados.

</div>

**b)** Boa para buscas por faixa de valor?

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

A B-tree (e especialmente a B+ tree) √© boa para buscas por faixa de valor porque as chaves ficam ordenadas, permitindo percorrer os n√≥s ou folhas em sequ√™ncia sem precisar recome√ßar a busca a cada valor.

</div>

**Exerc√≠cio 5**: Pense em situa√ß√µes onde o **particionamento vertical** √© ben√©fico, e onde √© problem√°tico.

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

üü¢ Ben√©fico quando:

H√° muitas colunas na tabela e diferentes consultas usam conjuntos distintos de colunas.

Exemplo: separar colunas raramente acessadas (como ‚Äúfoto‚Äù ou ‚Äúdescri√ß√£o longa‚Äù) para otimizar consultas que s√≥ precisam de campos b√°sicos.

üî¥ Problem√°tico quando:

As consultas precisam de muitas colunas ao mesmo tempo, pois o sistema precisa juntar v√°rias parti√ß√µes (joins), o que aumenta o custo de leitura.

</div>

**Exerc√≠cio 6**: Pense em situa√ß√µes onde o **particionamento horizontal** √© ben√©fico, e onde √© problem√°tico.

<div style="background-color: rgba(71, 85, 155, 0.2); padding: 10px; border-radius: 5px; border-left: 4px solid #475569; border: 1px solid rgba(71, 85, 105, 0.3);">

üü¢ Ben√©fico quando:

A tabela tem muitos registros e pode ser dividida por intervalos de valores (por exemplo, por regi√£o ou por data).

Isso melhora o desempenho e facilita o balanceamento de carga em m√∫ltiplos servidores.

üî¥ Problem√°tico quando:

√â necess√°rio consultar dados de v√°rias parti√ß√µes ao mesmo tempo, pois o banco precisa unir resultados de v√°rios lugares, aumentando a lat√™ncia.

</div>

## Conex√£o

Vamos fechar a conex√£o e finalizamos por hoje!

In [39]:
connection.close()

## Refer√™ncias
- OLIVEIRA, C. H. P, SQL: Curso Pr√°tico, Novatec, 2002 CAP 4
- SILBERSCHATZ, A.; KORTH, H. F.; SUDARSHAN, S. DATABASE SYSTEM CONCEPTS, SEVENTH EDITION CAP 4.6
- https://jasimabasheer.com/posts/btrees

## Gabarito

**<div id="gab_ex1">Exerc√≠cio 1</div>**


<style>
.box {
  padding: 10px;
  border-radius: 5px;
  border-left: 4px solid var(--accent);
  border: 1px solid var(--border);
  background-color: var(--bg);
}

/* Tema Claro */
@media (prefers-color-scheme: light) {
  .box-error {
    --accent: #ef4444; /* Red-500 */
    --border: rgba(239, 68, 68, 0.4);
    --bg: rgba(239, 68, 68, 0.15);
  }

  .box-warn {
    --accent: #f59e0b; /* Amber-500 */
    --border: rgba(245, 158, 11, 0.4);
    --bg: rgba(245, 158, 11, 0.15);
  }
}

/* Tema Escuro */
@media (prefers-color-scheme: dark) {
  .box-error {
    --accent: #f87171; /* Red-400 */
    --border: rgba(248, 113, 113, 0.4);
    --bg: rgba(248, 113, 113, 0.2);
  }

  .box-warn {
    --accent: #fbbf24; /* Amber-400 (mais claro no dark) */
    --border: rgba(251, 191, 36, 0.4);
    --bg: rgba(251, 191, 36, 0.2);
  }
}
</style>

<div class="box box-error">

**a)** Complexide de busca O(1) üòç

**b)** N√£o mant√©m rela√ß√£o de ordem üò≠
    
</div>